https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
!conda env list


# conda environments:
#
base                 * /home/janssen/mambaforge
postprocessing         /home/janssen/mambaforge/envs/postprocessing
preprocessing          /home/janssen/mambaforge/envs/preprocessing
processing             /home/janssen/mambaforge/envs/processing



# Preprocessing 

In [2]:
from nfdi_ing_workflow import (
    generate_mesh as _generate_mesh, 
    convert_to_xdmf as _convert_to_xdmf,
    poisson as _poisson,
    plot_over_line as _plot_over_line,
    substitute_macros as _substitute_macros,
    compile_paper as _compile_paper,
)

In [3]:
from pyiron_base import Project, job

In [4]:
generate_mesh = job(_generate_mesh)
convert_to_xdmf = job(_convert_to_xdmf, output_key_lst=["xdmf_file", "h5_file"])
poisson = job(_poisson, output_key_lst=["numdofs", "pvd_file", "vtu_file"])
plot_over_line = job(_plot_over_line)
substitute_macros = job(_substitute_macros)
compile_paper = job(_compile_paper)

In [5]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [6]:
domain_size = 2.0

## generate mesh

In [7]:
gmsh_output_file = generate_mesh(
    domain_size=domain_size,
    pyiron_project=pr
)

## convert to xdmf

In [8]:
meshio_output_dict = convert_to_xdmf(
    gmsh_output_file=gmsh_output_file,
    pyiron_project=pr,
)

# Processing

## poisson

In [9]:
poisson_dict = poisson(
    meshio_output_xdmf=meshio_output_dict.output.xdmf_file, 
    meshio_output_h5=meshio_output_dict.output.h5_file,
    pyiron_project=pr,
)

# Postprocessing

## plot over line

In [10]:
pvbatch_output_file = plot_over_line(
    poisson_output_pvd_file=poisson_dict.output.pvd_file, 
    poisson_output_vtu_file=poisson_dict.output.vtu_file,
    pyiron_project=pr,
)

## substitute macros

In [11]:
macros_tex_file = substitute_macros( 
    pvbatch_output_file=pvbatch_output_file, 
    ndofs=poisson_dict.output.numdofs, 
    domain_size=domain_size,
    pyiron_project=pr,
)

## compile paper

In [12]:
paper_output = compile_paper(
    macros_tex=macros_tex_file, 
    plot_file=pvbatch_output_file,
    pyiron_project=pr,
)

# Convert 

In [13]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [14]:
workflow_json_filename = "pyiron_base_nfdi.json"

In [15]:
write_workflow_json(delayed_object=paper_output, file_name=workflow_json_filename)

# Load Workflow with aiida

In [ ]:
from aiida import load_profile

load_profile()

In [ ]:
from python_workflow_definition.aiida import load_workflow_json

In [ ]:
wg = load_workflow_json(file_name=workflow_json_filename)
wg

In [ ]:
wg.run()

# Load Workflow with pyiron_base

In [16]:
from python_workflow_definition.jobflow import load_workflow_json

In [17]:
from jobflow.managers.local import run_locally

In [18]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [19]:
result = run_locally(flow)
result

2025-04-02 14:59:47,651 INFO Started executing jobs locally
2025-04-02 14:59:49,002 INFO Starting job - generate_mesh (c116d67e-4d2a-4e50-82af-faee6ee474ef)
2025-04-02 14:59:54,666 INFO Finished job - generate_mesh (c116d67e-4d2a-4e50-82af-faee6ee474ef)
2025-04-02 14:59:54,666 INFO Starting job - convert_to_xdmf (5283e8af-6407-4a02-8bb7-0057df7ec6fe)
2025-04-02 15:00:01,183 INFO Finished job - convert_to_xdmf (5283e8af-6407-4a02-8bb7-0057df7ec6fe)
2025-04-02 15:00:01,184 INFO Starting job - poisson (ce03af6b-866c-41ba-b17c-9bdf1e965e58)
2025-04-02 15:00:10,982 INFO Finished job - poisson (ce03af6b-866c-41ba-b17c-9bdf1e965e58)
2025-04-02 15:00:10,983 INFO Starting job - plot_over_line (582b8920-93f3-4a9e-b216-e3b30a22f1dd)
2025-04-02 15:00:21,672 INFO Finished job - plot_over_line (582b8920-93f3-4a9e-b216-e3b30a22f1dd)
2025-04-02 15:00:21,673 INFO Starting job - substitute_macros (596ecf51-bc63-4770-8f0f-802662bb46df)
2025-04-02 15:00:27,059 INFO Finished job - substitute_macros (596ecf

{'c116d67e-4d2a-4e50-82af-faee6ee474ef': {1: Response(output='/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.msh', detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd'))},
 '5283e8af-6407-4a02-8bb7-0057df7ec6fe': {1: Response(output={'xdmf_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.xdmf', 'h5_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/preprocessing/square.h5'}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd'))},
 'ce03af6b-866c-41ba-b17c-9bdf1e965e58': {1: Response(output={'numdofs': 357, 'pvd_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/processing/poisson.pvd', 'vtu_file': '/home/janssen/notebooks/2025/2025-04-02-nfdi-pwd/processing/poisson000000.vtu'}, detour=None, a